### Concepts regarding RSI

-   RS = GAIN EMA / LOSS EMA
where,
    - RS = Relative Strength
    - GAIN EMA = Exponential Moving Average of the gains
    - LOSS EMA = Exponential Moving Average of the losses

- RSI = 100.0 - (100.0 / (1.0 + RS))
where,
    - RSI = Relative Strength Index
    - RS = Relative Strength

- IF PREVIOUS RSI > 30 AND CURRENT RSI < 30 ==> BUY SIGNAL
- IF PREVIOUS RSI < 70 AND CURRENT RSI > 70 ==> SELL SIGNAL


#### Implementation 

1. Importing Packages
2. Extracting Data from BBG/yfinance
3. RSI calculation
4. RSI Plot
5. Creating the Trading Strategy
6. Plotting the Trading Lists
7. Creating our Position
8. Backtesting

In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from math import floor
import datetime
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [15]:
# my_asset_basket = ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'FB', 'NTES', 'ASML', 'AVGO', 'CSCO', 'PEP', 'COST', 'ADBE', 'CMCSA', 'INTC', 'AZN', 'QCOM', 'NFLX']
my_asset_basket = ['AAPL']
zscore = 'Adj Close: zscore'
def RSI_strategy(assets = my_asset_basket, starting_date = '2018-01-01', ending_date ='2022-01-01'):

    for asset in assets:
    
        my_assets = yf.download(asset, interval='1d', start=starting_date, end=ending_date)

        # data transforms
        lookback = 14
        my_assets[zscore] = (my_assets['Adj Close'] - my_assets['Adj Close'].rolling(lookback).mean()) / my_assets['Adj Close'].rolling(lookback).std()

        plt.title(f'{asset} Z-Score Adjusted Closing Prices')
        plt.ylabel('Price ($USD)')
        plt.xlabel('Date')
        plt.grid()
        plt.show();

# __________________________________________________________________________________________________________________________________________
# Function 1 - Calculate the RSI
        def get_rsi(df,asset, lookback):
            
'''
Delta is essentially defined as your price change between the current price and
the previous/subsequent price of the underlying secuity
'''
            df[asset + '_Delta'] = asset['Adj Close'].diff().dropna()

            gains = []
            loss = []

            for i in range(len(df[asset + '_Delta'])):
                if df[asset + '_Delta'][i]< 0:
                    gains.append(0)
                    loss.append(df[asset + '_Delta'][i])
                else:
                    gains.append(df[asset + '_Delta'][i])
                    loss.append(0)

            asset_gain_series = asset + '_gain_series'
            asset_loss_series = asset + '_loss_series'
            asset_gain_ewm = asset + 'gain_ewm'
            asset_loss_ewm = asset + 'loss_ewm'
            Relative_Strength = asset + '_RelativeStrength'
            RSI = asset + '_RSI'
            global RSI_14
            RSI_14 = asset + '_RSI_14'

            df[asset_gain_series] = gains
            df[asset_loss_series] = loss.abs()

            df[asset_gain_ewm] = gains.ewm(com= lookback - 1, adjust=False).mean()
            df[asset_loss_ewm] = loss.abs().ewm(com= lookback - 1, adjust=False).mean()

            df[Relative_Strength] = df[asset_gain_ewm] / df[asset_loss_ewm]
            df[RSI] = 100 - (100 / (1 + df[Relative_Strength]))

            # df[asset + 'RSI_14'] = pd.DataFrame(df[asset + '_RSI']).rename(columns={0:asset + '_RSI_14'}).set_index(delta.index) 

            df[RSI_14] = df[RSI_14].dropna()
            return df[RSI_14]


        assets[df[RSI_14]] = get_rsi(df,asset, lookback = 14)
        assets[df[RSI_14]] = my_assets[asset + '_RSI_14'].dropna()
        assets[df[RSI_14]]




# plot price
tsla['Adj Close'].plot(figsize = (15,6))
plt.title('TSLA Adjusted Close Prices')
plt.grid()
plt.show();

# plot corresponding RSI values and significant levels
tsla['RSI_14'].plot(figsize = (15,6), color='orange')
plt.title('TSLA Relative Strength Index')

plt.axhline(0, linestyle='--', alpha=0.1)
plt.axhline(30, linestyle='--')
plt.axhline(70, linestyle='--')
plt.axhline(100, linestyle='--', alpha=0.1)
plt.grid()
plt.show();

# ax1 = plt.subplot2grid(shape=(4,4), loc=(0,0))
# ax2 = plt.subplot2grid(shape=(4,4), loc=(3,0))

# ax1.plot(tsla['Adj Close'], linewidth = 2.5)
# ax2.plot(tsla['RSI_14'], color = 'orange', linewidth = 2.5)

# ax1.set_title('TSLA ADJUSTED CLOSE PRICE')
# ax2.set_title('TSLA RELATIVE STRENGTH INDEX')

# ax2.axhline(30, linestyle = '--', linewidth = 1.5)
# ax2.axhline(70, linestyle = '--', linewidth = 1.5)

# plt.show();

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 65)

#### Creating the RSI strategy function

In [ ]:
def implement_rsi_strategy(prices, rsi):
    long_price = []
    short_price = []
    rsi_signal = []
    signal = 0

    for i in range(len(rsi)):
        if rsi[i-1] > 30 and rsi[i] < 30:
            if signal!= 1:
                long_price.append(prices[i])
                short_price.append(np.nan)
                signal = 1
                rsi_signal.append(signal)

            else:
                long_price.append(np.nan)
                short_price.append(np.nan)
                rsi_signal.append(0)

        elif rsi[i-1] < 70 and rsi[i] > 70:
            if signal != 1:
                long_price.append(np.nan)
                short_price.append(prices[i])
                signal = -1
                rsi_signal.append(signal)

            else:
                long_price.append(np.nan)
                short_price.append(np.nan)
                rsi_signal.append(0)

        else:
            long_price.append(np.nan)
            short_price.append(np.nan)
            rsi_signal.append(0)

    return long_price, short_price, rsi_signal

long_price, short_price, rsi_signal = implement_rsi_strategy(tsla['Adj Close'], tsla['RSI_14'])


# ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 4, colspan = 1)
# ax2 = plt.subplot2grid((10,1), (5,0), rowspan = 4, colspan = 1)
# ax1.plot(tsla['Adj Close'], linewidth = 2.5, color = 'skyblue', label = 'TSLA')
# ax1.plot(tsla.index, long_price, marker = '^', ms = 10, color = 'green', label = 'BUY SIGNAL')
# ax1.plot(tsla.index, short_price, marker = 'v', ms = 10, color = 'red', label = 'SELL SIGNAL')
# ax1.set_title('TSLA RSI TRADE SIGNALS')
# ax2.plot(tsla['RSI_14'], color = 'orange', linewidth = 2.5)
# ax2.axhline(30, linestyle = '--', linewidth = 1.5)
# ax2.axhline(70, linestyle = '--', linewidth = 1.5)
# plt.show();

# PLOT SIGNALS

# plot price
tsla_plot = tsla['Adj Close']
tsla_rsi = tsla['RSI_14']
rsi_plot = sns.lineplot(x=tsla_rsi.index, y=tsla_rsi.values)
rsi_plot.set_title('Relative Strength Index of TSLA stock')
rsi_plot.set_ylabel('RSI Value')

tsla_plot.plot(figsize = (15,5), label = 'TSLA')

y1 = tsla_rsi.values > 70
tsla_rsi.plot(x=tsla_rsi.index,y= y1, marker = '^', color = 'green', ms=15, label='Long Signal') # long_price variable
tsla_rsi.plot(x=tsla_rsi.index, y=np.nan, marker = 'v', color = 'red', ms=15, label='Short Signal') # short_price variable
plt.title('TSLA Adjusted Close Prices')
plt.grid()
plt.show();

# plot corresponding RSI values and significant levels
tsla_rsi.plot(figsize = (15,5), color='orange', label = 'RSI')
plt.title('TSLA Relative Strength Index')

plt.axhline(0, linestyle='--', alpha=0.1)
plt.axhline(30, linestyle='--', color='green')
plt.axhline(70, linestyle='--', color='red')
plt.axhline(100, linestyle='--', alpha=0.1)
plt.grid()
plt.show();


### Creating our Position

In [ ]:
position = []
for i in range(len(rsi_signal)):
    if rsi_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(tsla['Adj Close'])):
    if rsi_signal[i] == 1:
        position[i] = 1
    elif rsi_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]

rsi = tsla['RSI_14']
close_price = tsla['Adj Close']
rsi_signal = pd.DataFrame(rsi_signal).rename(columns={0:'RSI_signal'}).set_index(tsla.index)
position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(tsla.index)

frames = [close_price, rsi, rsi_signal, position]
strategy = pd.concat(frames, join = 'inner', axis=1)

strategy.head()

### Final Component: Backtesting